In [19]:
%load_ext autoreload
%autoreload 2

import torch
import torchvision
from torch import nn
from torch.utils import data
from torchvision import transforms
from torchvision import models

import training_utils as utils
import network_utils as nutils

import distillation_methods_module

import numpy as np

if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

print(device)

torch.autograd.set_detect_anomaly(False)
torch.autograd.profiler.profile(False)
torch.autograd.profiler.emit_nvtx(False)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cuda:0


In [2]:
# Download the training and test sets
trans = transforms.ToTensor()
mnist_train = torchvision.datasets.CIFAR10(
    root="../data", train=True, transform=trans, download=True)
mnist_test = torchvision.datasets.CIFAR10(
    root="../data", train=False, transform=trans, download=True)

train_iter = data.DataLoader(mnist_train, batch_size=256, shuffle=True, num_workers=4, pin_memory=True)
test_iter = data.DataLoader(mnist_test, batch_size=256, shuffle=False, num_workers=4, pin_memory=True)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class Teacher_Net(nn.Module):
    def __init__(self, dropout=0.5):
        super().__init__()
        
        self.linear_1 = nn.Linear(784, 1200)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout)

        self.linear_2 = nn.Linear(1200, 1200)

        self.linear_3 = nn.Linear(1200, 1200)

        self.linear_4 = nn.Linear(1200, 10)
    
    def forward(self, input):
        input = torch.flatten(input, start_dim=1)
        out = self.linear_1(input)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.linear_2(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.linear_3(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.linear_4(out)

        return out

In [4]:
class Student_Net(nn.Module):
    def __init__(self, dropout=0.5):
        super().__init__()
        
        self.linear_1 = nn.Linear(784, 600)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout)

        self.linear_2 = nn.Linear(600, 1200)

        self.linear_3 = nn.Linear(1200, 1200)

        self.linear_4 = nn.Linear(1200, 10)
    
    def forward(self, input):
        input = torch.flatten(input, start_dim=1)
        out = self.linear_1(input)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.linear_2(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.linear_3(out)
        out = self.relu(out)
        out = self.dropout(out)

        out = self.linear_4(out)

        return out

In [3]:
class Teacher_Net(nn.Module):
    def __init__(self, dropout=0.5):
        super().__init__()
        
        self.linear_1 = nn.Conv2d(3, 32, (3,3))
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout)
        self.maxpool_1 = nn.MaxPool2d((2,2))

        self.linear_2 = nn.Conv2d(32, 64, (3,3))
        self.maxpool_2 = nn.MaxPool2d((2,2))

        self.linear_3 = nn.Conv2d(64, 64, (3,3))
        self.maxpool_3 = nn.MaxPool2d((2,2))

        self.linear_4 = nn.Linear(256, 10)
    
    def forward(self, input):
        out = self.linear_1(input)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.maxpool_1(out)

        out = self.linear_2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.maxpool_2(out)

        out = self.linear_3(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.maxpool_3(out)

        
        out = torch.flatten(out, start_dim=1)
        out = self.linear_4(out)

        return out

In [4]:
class Student_Net(nn.Module):
    def __init__(self, dropout=0.5):
        super().__init__()
        
        self.linear_1 = nn.Conv2d(3, 32, (3,3))
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=dropout)
        self.maxpool_1 = nn.MaxPool2d((2,2))

        self.linear_2 = nn.Conv2d(32, 64, (3,3))
        self.maxpool_2 = nn.MaxPool2d((2,2))

        self.linear_3 = nn.Conv2d(64, 64, (3,3))
        self.maxpool_3 = nn.MaxPool2d((2,2))

        self.linear_4 = nn.Linear(256, 10)
    
    def forward(self, input):
        out = self.linear_1(input)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.maxpool_1(out)

        out = self.linear_2(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.maxpool_2(out)

        out = self.linear_3(out)
        out = self.relu(out)
        out = self.dropout(out)
        out = self.maxpool_3(out)
        
        out = torch.flatten(out, start_dim=1)
        out = self.linear_4(out)

        return out

In [14]:
PATH = r"D:\Work\UNI\QMUL\Project\Models\ResNet152\CIFAR100_e250_a6172.tar"
num_classes = 100

teacher = models.resnet152(pretrained=False)
num_ftrs = teacher.fc.in_features
teacher.fc = nn.Linear(num_ftrs, num_classes)
optimizer = torch.optim.Adam(teacher.parameters(), lr=0.001)

checkpoint = torch.load(PATH)
teacher.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

teacher.to(device)
print("")

d:\Work\UNI\QMUL\Project\repo\DistillationComparison\.distenv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
d:\Work\UNI\QMUL\Project\repo\DistillationComparison\.distenv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [15]:
student = models.resnet101(pretrained=False)
num_ftrs = student.fc.in_features
student.fc = nn.Linear(num_ftrs, num_classes)

student.to(device)
print("")

In [ ]:
teacher = Teacher_Net().to(device)
num_epochs = 10
loss_fn = nn.CrossEntropyLoss(reduction='none').to(device)
optimizer = torch.optim.Adam(teacher.parameters(), lr=0.001)
history = utils.train(teacher, utils.train_epoch, train_iter, test_iter, loss_fn, num_epochs, optimizer)

In [ ]:
student = Student_Net().to(device)
optimizer = torch.optim.Adam(student.parameters(), lr=5e-3)

d_logits = distillation_methods_module.Logits_Distiller(5, teacher=teacher, student=student, optimizer=optimizer)
d_logits.train(train_iter, test_iter, num_epochs)

In [ ]:
num_epochs = 10

#student = Student_Net().to(device)
optimizer = torch.optim.Adam(student.parameters(), lr=1e-3)

hint_layer = nutils.get_layer_in_module_from_index(teacher.layer2[-1], -3)
guided_layer = nutils.get_layer_in_module_from_index(student.layer2[-1], -3)

d_features = distillation_methods_module.Features_Distiller(hint_layer=hint_layer, guided_layer=guided_layer, is_2D=True, teacher=teacher, student=student, optimizer=optimizer)
d_features.train(train_iter, test_iter, num_epochs)

In [11]:
num_epochs = 10

#student = Student_Net().to(device)
optimizer = torch.optim.Adam(student.parameters(), lr=1e-3)

hint_layer_1 = nutils.get_layer_in_module_from_index(teacher.layer2[0], -3)
hint_layer_2 = nutils.get_layer_in_module_from_index(teacher.layer2[-1], -3)
hint_layers = (hint_layer_1, hint_layer_2)

guided_layer_1 = nutils.get_layer_in_module_from_index(student.layer2[0], -3)
guided_layer_2 = nutils.get_layer_in_module_from_index(student.layer2[-1], -3)
guided_layers = (guided_layer_1, guided_layer_2)

d_relation = distillation_methods_module.Relations_Distiller(hint_layers=hint_layers, guided_layers=guided_layers, teacher=teacher, student=student, optimizer=optimizer)
d_relation.train(train_iter, test_iter, num_epochs)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (256x16384 and 8192x256)